## Import Libraries and Functions and Define the Model

In [3]:
# Physics Informed Neural Network with Taylor Series

from miscFun import *
output_notebook()

'''
N_INPUT: The number of input bio-z dimensions for one heartbeat
N_FEAT: The number of physiological features
N_EXT: The number of features extracted by the CNN
'''
#model_DNN 函数定义了三个参数：心跳数据的输入维度数量 N_INPUT，生理特征的数量 N_FEAT，以及CNN提取的特征数量 N_EXT。
def model_DNN(N_INPUT, N_FEAT=1, N_EXT=100):
    # The input to the model is a 1D tensor representing a time series of heartbeat data, sampled with 250/8 points for 30 seconds 
    inp_beat=tf.keras.Input(shape=(N_INPUT))
    
    # Define the 1D CNN for NN feature extraction
    # The input tensor is first expanded by one dimension (from 1D to 2D) to be compatible with the Conv1D layer
    cnn1_1 = tf.keras.layers.Conv1D(32,5,activation='relu')(tf.keras.backend.expand_dims(inp_beat,axis=-1))
    #tf.keras.layers.Conv1D(32, 5, activation='relu') 创建了一个一维卷积层，
    #其中：32 表示卷积层中滤波器的数量。5 是每个滤波器的尺寸，即卷积窗口的大小。activation='relu' 指定了激活函数为ReLU
    cnn1_2 = tf.keras.layers.Conv1D(64,3,activation='relu')(cnn1_1)
    #创建了第二个一维卷积层，与第一个卷积层类似，但滤波器数量增加到64，卷积窗口大小变为3。这允许网络在更细粒度上学习特征。
    mp_cnn1 = tf.keras.layers.MaxPooling1D(pool_size=3,strides=1)(cnn1_2)
    #创建了一个一维最大池化层，指定了池化窗口的大小为3，池化窗口移动的步长为1，这意味着池化操作将应用于每个可能的位置，以减少特征的空间维度，同时保留尽可能多的信息。
    fl_cnn1 = tf.keras.layers.Flatten()(mp_cnn1)
    #将经过卷积和池化操作后的二维张量展平为一维张量。这是为了准备将特征输入到后续的全连接层中进行进一步的处理。
    
    # A fully connected layer further processes the flattened tensor and extracts N_EXT features
    # 创建了一个全连接层，该层输出的节点数为N_EXT个，指定了使用ReLU激活函数，是将前一层的输出（fl_cnn）作为当前全连接层的输入
    feat_ext = tf.keras.layers.Dense(N_EXT,activation='relu')(fl_cnn1) 
    
    # Define physiological features (case study uses 3 features), each of these features is expected to be a 1D tensor
    # 这行注释说明了接下来的代码将定义三个生理特征输入，案例研究中使用了三个特征。
    inp_feat1 = tf.keras.Input(shape=(N_FEAT)) # feat 1 创建了一个输入层，用于接收第一个生理特征, shape=(N_FEAT) 指定了输入数据的形状
    inp_feat2 = tf.keras.Input(shape=(N_FEAT)) # feat 2
    inp_feat3 = tf.keras.Input(shape=(N_FEAT)) # feat 3
    
    # The extracted features and physiological features are concatenated together 
    #创建了一个拼接层，用于沿着指定的轴将多个输入张量拼接在一起。
    #这里包含了三个生理特征的输入层 inp_feat1、inp_feat2 和 inp_feat3，以及之前通过全连接层 feat_ext 提取的特征。这些输入被作为一个列表传递给 Concatenate 层。
    #feat_comb 包含了所有拼接在一起的特征，它将作为模型中下一个层的输入
    feat_comb = tf.keras.layers.Concatenate()([inp_feat1,inp_feat2,inp_feat3,feat_ext])
    
    # A fully connected layer with is applied to the concatenated features
    #全连接层 dense1_1，该层有60个节点，使用ReLU激活函数
    dense1_1 = tf.keras.layers.Dense(60,activation='relu')(feat_comb) 
    #输出层 out，N_FEAT 是输出层的节点数，与函数 model_DNN 的参数相对应，表示模型最终预测的生理特征的数量，dense1_1的输出作为输入
    out = tf.keras.layers.Dense(N_FEAT)(dense1_1) 
    
    # Finally, the model is instantiated with the specified inputs and outputs
    #创建了一个新的模型实例
    model = tf.keras.Model(inputs=[inp_beat, inp_feat1, inp_feat2, inp_feat3], outputs=[out])
    return model

Loading BokehJS ...

### Import a Demo Data

In [7]:
# load an example data for demo
# pd.read_pickle：这是Pandas库提供的函数，用于读取pickle格式的文件
# compression='gzip'：这个参数指定了文件压缩的类型。告诉 read_pickle 函数使用gzip压缩算法来解压缩pickle文件
df_demo_data = pd.read_pickle('data_demo_pinn_bioz_bp',compression='gzip')
print(df_demo_data.head())

   trial_id                                         bioz_beats  phys_feat_1  \
0         1  [4.674306050529548e-06, 3.873631220239295e-06,...     5.817552   
1         1  [4.806376660377724e-06, 4.00727594328936e-06, ...     6.002678   
2         1  [4.4183450820834e-06, 3.6333050096782107e-06, ...     5.632601   
3         1  [4.076650431880851e-06, 3.4283849875982737e-06...     5.119284   
4         1  [3.848732650901477e-06, 3.1940944086055694e-06...     4.833644   

   phys_feat_2  phys_feat_3         sys  
0   905.797101    97.832540  141.154438  
1   905.923964    99.785117  143.502124  
2   897.170462   100.450450  147.810844  
3   919.250190   102.752581  150.845861  
4   914.810008   102.519611  153.840739  


### Preprocess and Prepare the Train/Test Datasets

In [14]:
# Initialize a SEED value to ensure that the random processes in the code can be reproduced.
SEED = 123

# Call the function with seed value
set_global_determinism(seed=SEED)

# The keys for the beat data (beat_key), the target (out_key), and the features (feat_keys) are defined
beat_key = 'bioz_beats'
out_key = 'sys'
feat_keys = ['phys_feat_1','phys_feat_2','phys_feat_3']

# Data scaling of BP, input beats, and input features
# This scaler standardizes by removing the mean and scaling to unit variance
# This is done to ensure having the same scale, which can improve the performance of machine learning algorithms
# preprocessing.StandardScaler 是Scikit-learn库中用于数据标准化的类。Standardization。
# to_numpy() 方法将Pandas Series转换为NumPy数组，[:, None] 确保数据是二维数组的形式，即使只有一列数据。
scaler_out = preprocessing.StandardScaler().fit(df_demo_data[out_key].to_numpy()[:, None]) 
# 标准化心跳数据
scaler_beats = preprocessing.StandardScaler().fit(np.concatenate(df_demo_data[beat_key].to_numpy())[:, None])
# 标准化特征数据
scaler_X = [preprocessing.StandardScaler().fit(df_demo_data[a].to_numpy()[:, None]) for a in feat_keys]

# Apply Scaling
# The scaled versions of the BP, input beats, and input features are then added to the dataframe
df_demo_data.loc[df_demo_data.index,beat_key+'_scaled'] = df_demo_data.apply(lambda x: np.concatenate(scaler_beats.transform(x[beat_key][:, None])), axis=1).to_numpy()   
df_demo_data.loc[df_demo_data.index,out_key+'_scaled'] = df_demo_data.apply(lambda x: np.concatenate(scaler_out.transform(np.array([x[out_key]])[:, None]))[0], axis=1).to_numpy()
for tmp_key, tmp_count in zip(feat_keys, range(len(feat_keys))):
    df_demo_data.loc[df_demo_data.index, tmp_key+'_scaled'] = df_demo_data.apply(lambda x: np.concatenate(scaler_X[tmp_count].transform(np.array([x[tmp_key]])[:, None])), axis=1).to_numpy()

# Fetch scaled feature names
X_keys = [a+'_scaled' for a in feat_keys]

# 查看标准化后的心跳数据的前5行
print("标准化后的心跳数据前5行:")
print(df_demo_data[beat_key + '_scaled'].head())

# 查看标准化后的目标变量的前5行
print("\n标准化后的目标变量前5行:")
print(df_demo_data[out_key + '_scaled'].head())

# 查看所有标准化后的生理特征的前5行
print("\n标准化后的生理特征前5行:")
for key in X_keys:  # X_keys 包含了所有标准化特征的键名
    print(f"{key}: ")
    print(df_demo_data[key].head())
    print()

# Prepare train/test using minimal training the BP
# Fetch data shapes
# 获取序列数据的长度
length_seq_x = df_demo_data.apply(lambda x: len(x[beat_key+'_scaled']), axis=1).unique()[0]
# Set the length of the target to 1
# 设置了目标变量（血压 BP）的长度为1。这通常意味着每个心跳序列对应的是一个单独的血压值，或者模型的输出是一个标量值。
length_seq_y = 1

# Start with all points
# Reshape the scaled beat data into a 2D array where each row corresponds to a sample and each column corresponds to a time point in the beat sequence
# The same is done for the features and the target
# 这行代码首先使用 np.concatenate 将DataFrame中所有行的标准化心跳数据（beat_key+'_scaled'）合并成一个一维数组。然后，使用 np.reshape 将这个一维数组重塑为一个二维数组
all_beats = np.reshape(np.concatenate(df_demo_data[beat_key+'_scaled'].values), (len(df_demo_data), length_seq_x))
# 将所有生理特征的标准化数据合并成一个二维数组
[all_feat1, all_feat2, all_feat3] = [df_demo_data[a].values[:, None] for a in X_keys]
# 将所有目标变量的标准化数据合并成一个二维数组
all_out = df_demo_data[out_key+'_scaled'].values[:, None]

# Used only for plotting purposes
# 计算标准化后的目标变量的最大值和最小值
#out_max_rescaled = np.concatenate(scaler_out.inverse_transform(all_out[:, 0][:, None])).max()
#out_min_rescaled = np.concatenate(scaler_out.inverse_transform(all_out[:, 0][:, None])).min()

# 逆转换 all_out 以获取原始尺度的值
original_out = scaler_out.inverse_transform(all_out)

# 计算逆转换后数组的最大值和最小值
out_max_rescaled = original_out.max()
out_min_rescaled = original_out.min()

# Given different trials have time gaps, ignore first 3 instances from indices to prevent discontiunity in training
# 为了避免训练数据中的不连续性，忽略了每个试验的前三个数据点
list_all_length = [0]
for _, df_tmp in df_demo_data.groupby(['trial_id']):
    list_all_length.append(len(df_tmp))
ix_ignore_all = np.concatenate(np.array([np.arange(a, a+3,1) for a in list(np.cumsum(list_all_length)[:-1])]))

# Update the final indices set
# 将所有数据点的索引分为训练集和测试集
ix_all=list(set(np.arange(len(df_demo_data)))-set(ix_ignore_all))

# Separate train/test based on minimal training criterion
# 使用随机种子将数据集分为训练集和测试集，这行代码设置了随机数生成器的种子，以确保数据集的划分是可重复的
random.seed(0)
bp_dist = df_demo_data[out_key].values

# Find indices for train and test datasets
# The target values are sorted in ascending order, and the sorted indices are split into multiple subsets
# For each subset, a random index is selected as a training index
# np.argsort(bp_dist) 根据目标变量 bp_dist（即 df_demo_data[out_key].values）的值进行排序，并返回排序后的索引数组。
# np.split 将排序后的索引数组分割为多个子数组，每个子数组的长度由 np.histogram 函数计算得到。
ix_split = np.split([a for a in np.argsort(bp_dist) if a not in set(ix_ignore_all)], np.cumsum(np.histogram(bp_dist[ix_all],bins=np.arange(bp_dist[ix_all].min(), bp_dist[ix_all].max(), 1))[0]))
# 从每个子数组中随机选择一个索引作为训练集的索引，如果子数组为空，则选择-1作为索引，表示没有数据点被选择
ix_train = [random.Random(4).choice(a) if len(a)>0 else -1 for a in ix_split]
ix_train = list(set(ix_train)-set([-1]))

# Test set is all remaining points not used for training
# 将所有数据点的索引分为训练集和测试集
ix_test = list(set(ix_all) - set(ix_train))

# Build train and test datasets based on the indices
# 使用索引将数据集分为训练集和测试集
train_beats = all_beats[ix_train, :]
test_beats = all_beats[ix_test, :]
[train_feat1, train_feat2, train_feat3] = [all_feat1[ix_train, :], all_feat2[ix_train, :], all_feat3[ix_train, :]]
[test_feat1, test_feat2, test_feat3] = [all_feat1[ix_test, :], all_feat2[ix_test, :], all_feat3[ix_test, :]]
train_out = all_out[ix_train, :]
test_out = all_out[ix_test, :]

# 查看训练集和测试集的形状
print(f"训练集的形状: {train_beats.shape}")
print(f"测试集的形状: {test_beats.shape}")

# 输出训练集中的心跳数据的前5行
print("训练集中的心跳数据前5行:")
print(train_beats[:5])

# 输出测试集中的心跳数据的前5行
print("\n测试集中的心跳数据前5行:")
print(test_beats[:5])

# 输出训练集中的特征数据的前5行
print("\n训练集中的特征数据前5行:")
print(train_feat1[:5])

# 输出训练集中的目标变量的前5行
print("\n训练集中的目标变量前5行:")
print(train_out[:5])

# 输出测试集中的特征数据的前5行
print("\n测试集中的特征数据前5行:")
print(test_feat1[:5])




标准化后的心跳数据前5行:
0    [2.9279891461837044, 2.4172478429202755, 0.969...
1    [3.012235475460972, 2.5024982809790393, 1.0585...
2    [2.764714576204573, 2.2639465059145487, 0.8654...
3    [2.5467514726318945, 2.1332303709962863, 0.887...
4    [2.401365330729425, 1.9837790942355338, 0.7758...
Name: bioz_beats_scaled, dtype: object

标准化后的目标变量前5行:
0   -0.952151
1   -0.812206
2   -0.555365
3   -0.374448
4   -0.195925
Name: sys_scaled, dtype: float64

标准化后的生理特征前5行:
phys_feat_1_scaled: 
0      [1.048386639332269]
1     [1.1888416577787004]
2     [0.9080649136981447]
3     [0.5186121459215011]
4    [0.30189754244438355]
Name: phys_feat_1_scaled, dtype: object

phys_feat_2_scaled: 
0    [-0.09849125703922874]
1    [-0.09499063973369752]
2    [-0.33653323381721845]
3     [0.27273092140839417]
4     [0.15020931571389476]
Name: phys_feat_2_scaled, dtype: object

phys_feat_3_scaled: 
0     [1.145099680369798]
1     [1.476112018152463]
2     [1.588903218155048]
3    [1.9791739417836047]
4     [1.939679

In [9]:
#### Define model input tensors
# The training, testing, and all data are converted to TensorFlow tensors
# The tensors for the different datasets are grouped into lists 

model_inp = tf.convert_to_tensor(train_beats, dtype=tf.float32)
feat1_inp = tf.convert_to_tensor(train_feat1, dtype=tf.float32)
feat2_inp = tf.convert_to_tensor(train_feat2, dtype=tf.float32)
feat3_inp = tf.convert_to_tensor(train_feat3, dtype=tf.float32)
inp_comb = [model_inp, feat1_inp, feat2_inp, feat3_inp]

model_inp_test = tf.convert_to_tensor(test_beats, dtype=tf.float32)
feat1_inp_test = tf.convert_to_tensor(test_feat1, dtype=tf.float32)
feat2_inp_test = tf.convert_to_tensor(test_feat2, dtype=tf.float32)
feat3_inp_test = tf.convert_to_tensor(test_feat3, dtype=tf.float32)
inp_comb_test = [model_inp_test, feat1_inp_test, feat2_inp_test, feat3_inp_test]

model_inp_all = tf.convert_to_tensor(all_beats, dtype=tf.float32)
feat1_inp_all = tf.convert_to_tensor(all_feat1, dtype=tf.float32)
feat2_inp_all = tf.convert_to_tensor(all_feat2, dtype=tf.float32)
feat3_inp_all = tf.convert_to_tensor(all_feat3, dtype=tf.float32)
inp_comb_all = [model_inp_all, feat1_inp_all, feat2_inp_all, feat3_inp_all]

2024-06-13 10:58:06.706915: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-06-13 10:58:06.706961: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-06-13 10:58:06.706971: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-06-13 10:58:06.707034: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-13 10:58:06.707071: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Train the Conventional model

In [10]:
#############################
###### Conventional model
#############################

# A Deep Neural Network model is initialized with the dimension of the beats, the diemnsion of each feature, and the number of neurons in the first dense layer
model_dnn_conv = model_DNN(np.shape(train_beats)[-1], 1, 64)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)

# Two lists are initialized to keep track of the training and testing loss during each epoch
loss_list_conv = []
test_loss_list_conv = []

epochs = 5000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        
        tape.watch(inp_comb)      
        # Traditional out
        yh = model_dnn_conv(inp_comb, training=True)
        loss_ini = yh - train_out
        loss = K.mean(K.square(loss_ini))
        
    grads = tape.gradient(loss, model_dnn_conv.trainable_weights)

    loss_list_conv.append(float(loss))
    loss_final = np.min(loss_list_conv)
    optimizer.apply_gradients(zip(grads, model_dnn_conv.trainable_weights))

    pred_out = model_dnn_conv(inp_comb_test)
    
    test_loss_ini = pred_out - test_out
    test_loss = K.mean(K.square(test_loss_ini))
    test_loss_list_conv.append(float(test_loss))
    
    # If the training loss reaches a minimum value of 0.01, or the maximum number of epochs is reached, the training process is stopped
    if (loss_final<=0.01) | (epoch==epochs-1):
        print("Conventional model training Completed. Epoch %d/%d -- loss: %.4f" % (epoch, epochs, float(loss)))
        break

Conventional model training Completed. Epoch 342/5000 -- loss: 0.0099


### Train the PINN model

In [11]:
#############################
############### PINN MODEL
#############################

# A Deep Neural Network model is initialized with the dimension of the beats, the diemnsion of each feature, and the number of neurons in the first dense layer
model_dnn_pinn = model_DNN(np.shape(train_beats)[-1], 1, 64)
optimizer = tf.keras.optimizers.Adam(learning_rate=10e-4)

# Two lists are initialized to keep track of the training and testing loss during each epoch
loss_list_pinn = []
test_loss_list_pinn = []

epochs = 5000
for epoch in range(epochs):
    with tf.GradientTape() as tape:

        tape.watch(inp_comb)      
        # Traditional out
        yh = model_dnn_pinn(inp_comb, training=True)
        loss_ini = yh - train_out
        loss = K.mean(K.square(loss_ini))        
        
        # Additional tf.GradientTape contexts are used to compute the derivatives of the model's predictions with respect to the features 
        with tf.GradientTape() as deriv_f1:
            deriv_f1.watch(inp_comb_all)
            yhp = model_dnn_pinn(inp_comb_all, training=True)
        dx_f1 = deriv_f1.gradient(yhp, feat1_inp_all)

        with tf.GradientTape() as deriv_f2:
            deriv_f2.watch(inp_comb_all)
            yhp = model_dnn_pinn(inp_comb_all, training=True)
        dx_f2 = deriv_f2.gradient(yhp, feat2_inp_all)

        with tf.GradientTape() as deriv_f3:
            deriv_f3.watch(inp_comb_all)
            yhp = model_dnn_pinn(inp_comb_all, training=True)
        dx_f3 = deriv_f3.gradient(yhp, feat3_inp_all)

        # A physics-based prediction is computed by adding the model's predictions to the product of the computed derivatives and
        # the differences in the feature values between consecutive timesteps
        pred_physics = (yhp[:-1, 0]
                        +Multiply()([dx_f1[:-1, 0], feat1_inp_all[1:, 0] - feat1_inp_all[:-1, 0]])
                        +Multiply()([dx_f2[:-1, 0], feat2_inp_all[1:, 0] - feat2_inp_all[:-1, 0]])
                        +Multiply()([dx_f3[:-1, 0], feat3_inp_all[1:, 0] - feat3_inp_all[:-1, 0]])
                        )

        physics_loss_ini = pred_physics - yhp[1:, 0]
        physics_loss = K.mean(K.square(tf.gather_nd(physics_loss_ini,indices = np.array(ix_all[:-1])[:, None])))
    
        # The total loss is computed as the sum of the initial loss and ten times the physics-based loss
        # The physics-based loss is multiplied by a factor of ten to emphasize its importance in the loss function
        loss_total = loss + physics_loss * 10

    grads = tape.gradient(loss_total, model_dnn_pinn.trainable_weights)

    loss_list_pinn.append(float(loss))
    loss_final=np.min(loss_list_pinn)
    optimizer.apply_gradients(zip(grads, model_dnn_pinn.trainable_weights))
    
    pred_out = model_dnn_pinn(inp_comb_test)
    test_loss_ini = pred_out - test_out
    test_loss = K.mean(K.square(test_loss_ini))
    test_loss_list_pinn.append(float(test_loss))
    
    # If the training loss reaches a minimum value of 0.01, or the maximum number of epochs is reached, the training process is stopped
    if (loss_final<=0.01) | (epoch==epochs-1):
        print("PINN model training Completed. Epoch %d/%d -- loss: %.4f" % (epoch,epochs,float(loss)))
        break

PINN model training Completed. Epoch 2124/5000 -- loss: 0.0099


In [12]:
#The trained model's predictions on the test dataset are computed
pred_out = model_dnn_conv(inp_comb_test)

#The Pearson correlation coefficient and the Root Mean Square Error are calculated between the actual and predicted test outcomes
corr_conv = np.corrcoef(np.concatenate(test_out)[:], np.concatenate(pred_out)[:])[0][1]
rmse_conv = np.sqrt(np.mean(np.square
                           (np.concatenate(scaler_out.inverse_transform(np.concatenate(test_out)[:][:, None]))-
                            np.concatenate(scaler_out.inverse_transform(np.concatenate(pred_out)[:][:, None])))))

pred_out = model_dnn_pinn(inp_comb_test)
corr_pinn = np.corrcoef(np.concatenate(test_out)[:], np.concatenate(pred_out)[:])[0][1]
rmse_pinn = np.sqrt(np.mean(np.square(
    np.concatenate(scaler_out.inverse_transform(np.concatenate(test_out)[:][:, None]))-
    np.concatenate(scaler_out.inverse_transform(np.concatenate(pred_out)[:][:, None])))))

print('#### Conventional Performance ####')
print('Corr: %.2f,  RMSE: %.1f'%(corr_conv, rmse_conv))
print('----------------------------------')
print('#### PINN Performance ####')
print('Corr: %.2f,  RMSE: %.1f'%(corr_pinn, rmse_pinn))

#### Conventional Performance ####
Corr: 0.86,  RMSE: 10.2
----------------------------------
#### PINN Performance ####
Corr: 0.95,  RMSE: 5.3


In [15]:
s=figure(width=770,height=400,y_range=(out_min_rescaled-20,out_max_rescaled+20))
s.scatter(ix_test,np.concatenate(scaler_out.inverse_transform(np.concatenate(model_dnn_conv(inp_comb_test))[:,None])),size=7,line_color=None,color=palettes.Colorblind8[5],legend_label='Conv.')
s.scatter(ix_test,np.concatenate(scaler_out.inverse_transform(np.concatenate(model_dnn_pinn(inp_comb_test))[:,None])),size=7,line_color=None,color=palettes.Colorblind8[3],legend_label='PINN')
s.line(range(len(df_demo_data)),np.concatenate(scaler_out.inverse_transform(all_out[:,0][:,None])),line_width=3,line_color='black',line_alpha=1,line_dash='dashed',legend_label='True BP')
s.xaxis.axis_label='Beat time (s)'
s.yaxis.axis_label='SBP (mmHg)'

s2 = figure(width=500,height=400,y_axis_type="log",y_range=(1e-2,2))
#s2.line(np.linspace(0,100,len(loss_list_conv)),loss_list_conv,line_width=3,color='black',legend_label='PINN-test')
#s2.line(np.linspace(0,100,len(test_loss_list_conv)),test_loss_list_conv,line_width=3,line_dash='dashed',color='black',legend_label='Conv-test')
#s2.line(np.linspace(0,100,len(loss_list_pinn)),loss_list_pinn,line_width=3,alpha=0.8,color='orange',legend_label='PINN-train')
#s2.line(np.linspace(0,100,len(test_loss_list_pinn)),test_loss_list_pinn,line_width=3,line_dash='dashed',color='orange',legend_label='Conv-train')
# 假设您想要展示的训练百分比范围是从0到100，总共有len(loss_list_conv)个点

# Convert range object to list
x_range = list(range(len(loss_list_conv)))

# 更新图表绘制代码
s2.line(x_range, loss_list_conv, line_width=3, color='black', legend_label='PINN-test')
s2.line(x_range, test_loss_list_conv, line_width=3, line_dash='dashed', color='black', legend_label='Conv-test')
s2.line(x_range, loss_list_pinn, line_width=3, alpha=0.8, color='orange', legend_label='PINN-train')
s2.line(x_range, test_loss_list_pinn, line_width=3, line_dash='dashed', color='orange', legend_label='Conv-train')

s2.xaxis.axis_label = 'Training percent (%)'
s2.yaxis.axis_label = 'mse norm.'

figure_settings(s)
figure_settings(s2)

show(row(s,s2))


SerializationError: can't serialize <class 'range'>

In [23]:
plot = list(row(s,s2))
print (plot)
print (type(plot))



TypeError: 'Row' object is not iterable